In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import keras
#modelI=keras.models.load_model('modelnew.keras')
modelI=keras.models.load_model('saved_model/model.keras')

In [253]:

labels_symbols = {
    10:'-', 11:'+', 12:'*', 13:'=', 14:'a', 15:'b', 16:'<', 17:'>', 18:'/',19:'√', 20:'(',21:')'
}
for i in range(0,10):
    labels_symbols[i] = chr(48+i)

symbols_folder = {'*':'times','/':'','<':'','>':'','√':'','(':'',')':''}

In [237]:
import cv2

def get_parts_from_image(path, display_img = False ):
    img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    if display_img:
        print(path)
        plt.axis('off')
        plt.imshow(~img,cmap='gray')
        plt.show()

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    train_data=[]
    if img is not None:
        img=~img
        ret,thresh=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
        ctrs,ret=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        cnt=sorted(ctrs,key=lambda ctr:cv2.boundingRect(ctr)[0])
        w=int(28)
        h=int(28)
        rects=[]
        for c in cnt:
            x,y,w,h=cv2.boundingRect(c)
            rect=[x,y,w,h]
            rects.append(rect)
    #     print("rects", rects)
        bool_rect=[]
        for r in rects:
            l=[]
            for rec in rects:
                flag=0
                if rec!=r:
                    if r[0]<(rec[0]+rec[2]+10) and rec[0]<(r[0]+r[2]+10) and r[1]<(rec[1]+rec[3]+10) and rec[1]<(r[1]+r[3]+10):
                        flag=1
                    l.append(flag)
                if rec==r:
                    l.append(0)
            bool_rect.append(l)
    #     print("bools",bool_rect)
        dump_rect=[]
        for i in range(0,len(cnt)):
            for j in range(0,len(cnt)):
                if bool_rect[i][j]==1:
                    area1=rects[i][2]*rects[i][3]
                    area2=rects[j][2]*rects[j][3]
                    if(area1==min(area1,area2)):
                        dump_rect.append(rects[i])
    #     print("dump_rects",dump_rect)
        final_rect=[i for i in rects if i not in dump_rect]
    #     print("Final_rects",final_rect)
        for r in final_rect:
            x=r[0]
            y=r[1]
            w=r[2]
            h=r[3]
            im_crop=thresh[y:y+h+10,x:x+w+10]
            im_resize=cv2.resize(im_crop,(28,28))
    #         cv2.imshow("work",im_resize)
            train_data.append(im_resize)
        
        return train_data
    

In [251]:
import numpy as np
    
def recognise_parts(imgs, symb_imgs_store=None):
    s=""
    for i in range(len(imgs)):
        if symb_imgs_store is not None:
            symb_imgs_store.append(imgs[i])
        imgs[i]=np.array(imgs[i])
        imgs[i]=imgs[i].reshape(1,28,28,1)
    #     result=modelI.predictclasses(train_data[i])
        result=np.argmax(modelI.predict(imgs[i]), axis=1)
        s+= labels_symbols[result[0]]
    return s

In [258]:

def predict_image(img_path,display_parts=False,symb_imgs_store=None):
    train_data = get_parts_from_image(img_path, display_img = False)
    if display_parts:
        print(f"number of parts = {len(train_data)}")
    plt.rcParams["figure.figsize"] = (20,2)
    n_per_line = 10
    if display_parts:
        for i,part in enumerate(train_data):
            if i%n_per_line == 0 :
                fig, ax = plt.subplots(ncols = n_per_line)
                for x in ax: x.axis('off')
            ax[i%n_per_line].imshow(part, cmap='gray')
            #ax.plot()
        plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"] 
        plt.show()

    detected_exp = recognise_parts(train_data, symb_imgs_store = symb_imgs_store)

    if display_parts: 
        print(f"Detected Expression : {detected_exp}")
    
    return detected_exp


In [43]:
import pyscreenshot as IG
from datetime import datetime
import cv2
import numpy as np
def take_screenshot(winfo=None):
    imageName = "hellohi.jpg"
    screenshot = IG.grab()
    if winfo is not None:
        screenshot=screenshot.crop(winfo)
    screenshot_npArray = np.array(screenshot.getdata())
    filepath = f"./test_drawn_images/{imageName}"
    screenshot.save(filepath)
    return filepath


In [265]:
from tkinter import *
from tkinter import messagebox
from PIL import Image
import time
import os

In [ ]:
error_test_store = './test_images_error_to_train'

class paintApp:
    
    left_bt_up, right_bt_up = None, None
    x_pos, y_pos = None, None
    x1, y1, x2, y2 = None, None, None, None
    
    result_box = None
    store_for_train_box = None
    
    root = None
    #button
    
    def __init__(self,root):
        
        self.error_imgs_train_cache = []
        
        self.root = root
        self.cached_img_path=None
        
        drawing_area = Canvas(root, background='white')
        self.drawing_area = drawing_area
        #drawing_area.pack()
        drawing_area.grid(sticky=(W,E))
        drawing_area.columnconfigure(0,weight=1)
        
        drawing_area.bind("<Motion>", self.motion)
        drawing_area.bind("<ButtonPress-1>", self.left_bt_down)
        drawing_area.bind("<ButtonRelease-1>", self.left_bt_up)
        
        
        buttons_area = Canvas(root,background="red")
        #buttons_area.pack()
        buttons_area.grid(row=1,sticky=(W,E))
        buttons_area.columnconfigure(tuple(range(5)), weight=1)
        buttons_area.columnconfigure(tuple(range(2)), weight=4)
        
        self.result_box = Text(buttons_area, bg='black', fg='white',
                               height=0.5, width=10, font=('Helvatical bold',20), 
                               padx = 10, pady=10)
        #self.result_box['state'] = 'disabled'
        self.result_box.tag_configure("center", justify='center')
        self.result_box.tag_add("center", 1.0, "end")
        self.result_box.grid(row=0,column=1)#pack()
        
        submit_btn = Button(buttons_area, text="Predict", command = self.predict)
        submit_btn.grid(row=0,column=2)#pack()
        
        clear_btn = Button(buttons_area, text="clear", command = self.clear)
        clear_btn.grid(row=0,column=3)#pack()
        
        store_for_train_btn = Button(buttons_area, text="store for training", command = self.store_for_train)
        store_for_train_btn.grid(row=1,column=2)#pack()
        
        self.store_for_train_box = Text(buttons_area, bg='black', fg='white',
                               height=0.5, width=10, font=('Helvatical bold',20), 
                               highlightthickness = 0, borderwidth=2, 
                                insertwidth=1, insertbackground='green',
                               padx = 10, pady=10)
        self.store_for_train_box.tag_configure("center", justify='center')
        self.store_for_train_box.tag_add("center", 1.0, "end")
        self.store_for_train_box.grid(row=1,column=1)#pack()
        
        
    def left_bt_down(self, event=None):
        self.left_bt_up = False
        self.x1, self.y1 = event.x, event.y
    
    def left_bt_up(self, event=None):
        self.left_bt_up = True
        self.x2, self.y2 = event.x, event.y
        self.x_pos, self.y_pos = None, None
    
    def motion(self, event=None):
        self.pencil_draw(event)
        
    def pencil_draw(self, event=None):
        if self.left_bt_up == False:
            if None not in (self.x_pos, self.y_pos):
                self.drawing_area.create_line(self.x_pos,self.y_pos,event.x,event.y,
                                              width=3,fill='black',capstyle=ROUND,smooth=True)
                #event.widget.create_line(self.x_pos, self.y_pos, event.x, event.y, smooth = True, width=10)
            self.x_pos, self.y_pos = event.x, event.y
    
    def clear(self):
        self.cached_img_path = None
        self.drawing_area.delete('all')
        self.result_box.delete('1.0',END)
    
    def predict(self, event=None):
        
        #this part is for taking the screenshot and cropping the image to canvas only
        #needed to multiply by factors due to, i think, resolution mismatch
        left,top = self.drawing_area.winfo_rootx(),\
                    self.drawing_area.winfo_rooty()#+self.drawing_area.winfo_y()
        width,height = self.drawing_area.winfo_width(),self.drawing_area.winfo_height()
        res_fac = 1.2508,1.2531
        delta = 3,3
        delta_sz = 2,2
        left,top, width, height = (left+delta[0])*res_fac[0], (top+delta[1])*res_fac[1], \
                                    (width-delta[0]-delta_sz[0])*res_fac[0], (height-delta[1]-delta_sz[1])*res_fac[1]
        img_path = take_screenshot((left, top, left+width, top+height))
        
#         img=cv2.imread(img_path,cv2.IMREAD_UNCHANGED)
#         plt.axis('off')
#         plt.imshow(img)
#         plt.show()
        self.cached_img_path=img_path
        self.error_imgs_train_cache = []
        ans = predict_image(img_path, symb_imgs_store = self.error_imgs_train_cache)
        self.result_box.delete('1.0',END)
        self.result_box.insert('1.0',str(ans),"center")
        
    def store_for_train(self,event=None):
        #copy image to folder
            #create if not exists appropriate sub_folder and filename
        correct_expr = self.store_for_train_box.get('1.0',END)
        correct_expr = correct_expr.strip()
        print(len(correct_expr), len(self.error_imgs_train_cache))
        for i in range(len(correct_expr)):
            c = correct_expr[i]
            img = Image.fromarray(self.error_imgs_train_cache[i])
            path = error_test_store + '/'+ c if not c in symbols_folder else symbols_folder[c]
            print(path)
            if not os.path.isdir(path):
                os.mkdir(path)
            
            img_path = path + "/"+time.strftime("%Y%m%d-%H%M%S")+ '.jpg'
            img.save(img_path)
            
        pass
        
        
root = Tk()
paintApp(root)
root.mainloop()

1 1
./test_images_error_to_train/1
